In [1]:
import os
import sys
sys.path.append("../")
from Attune_function6_22 import *
from bgi.metrics.metrics import *
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt

In [2]:
#Select an available GPU to run on a multi-GPU computer or you can run it directly on the CPU without executing this cell
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = '6'
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
# input
adata_rna = sc.read('./data/greenleaf-final-RNA_wox_filter_hvg_rm_dc1r3_r1.h5ad')
adata_atac = sc.read('./data/greenleaf-final-ATAC_filter_rm_dc1r3_r1.h5ad')

# make TF-record

In [15]:
# make tfrecord
save_path = './tfrecord/RNA/'
sim_tf_path = concerto_make_tfrecord(adata_rna, tf_path = save_path, batch_col_name='Sample.ID')
save_path = './tfrecord/ATAC/'
sim_tf_path = concerto_make_tfrecord(adata_atac, tf_path = save_path, batch_col_name='Sample.ID')

AnnData object with n_obs × n_vars = 4733 × 2000
    obs: 'Cell.Barcode', 'Dissociation.ID', 'RNA.Counts', 'RNA.Features', 'RNA_snn_res.0.5', 'Sample.Age', 'Sample.Batch', 'Sample.ID', 'batch', 'cells', 'celltype', 'nCount_ambiguous', 'nCount_spliced', 'nCount_unspliced', 'nFeature_ambiguous', 'nFeature_spliced', 'nFeature_unspliced', 'percentMT', 'percentRibo', 'seurat_clusters', 'n_genes'
    var: 'blockCount', 'blockSizes', 'blockStarts', 'chrom', 'chromEnd', 'chromStart', 'gene_ids', 'gene_name', 'gene_type', 'havana_gene', 'hgnc_id', 'itemRgb', 'n_counts', 'name', 'score', 'strand', 'tag', 'thickEnd', 'thickStart', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'hvg'
counter: 1000 shape: (2000,), batch: 0
counter: 2000 shape: (2000,), batch: 0
counter: 3000 shape: (2000,), batch: 1
counter: 4000 shape: (2000,), batch: 1
AnnData object with n_obs × n_vars = 4733 × 41300
    obs: 'Cell.Bar

# train

In [16]:
random.seed(0)
RNA_tf_path = f'./tfrecord/RNA/'
ATAC_tf_path = f'./tfrecord/ATAC/'
dict_ = {'batch_size': 128, 'epoch_pretrain': 20, 'lr': 1e-4, 'drop_rate': 0.1,'temperature':0.1}
weight_path = f'./weight/pretrain/'
attune_pretrain(RNA_tf_path,ATAC_tf_path, weight_path,super_parameters = dict_)


./tfrecord/RNA/tf_1.tfrecord
./tfrecord/ATAC/tf_1.tfrecord
Epoch1, step5, total loss:13.431
Epoch1, step10, total loss:12.588
Epoch1, step15, total loss:12.151
./tfrecord/RNA/tf_0.tfrecord
./tfrecord/ATAC/tf_0.tfrecord
Epoch1, step5, total loss:11.051
Epoch1, step10, total loss:10.965
Epoch1, step15, total loss:10.880
Epoch1, step20, total loss:10.795
./tfrecord/RNA/tf_1.tfrecord
./tfrecord/ATAC/tf_1.tfrecord
Epoch2, step5, total loss:10.180
Epoch2, step10, total loss:10.094
Epoch2, step15, total loss:10.010
./tfrecord/RNA/tf_0.tfrecord
./tfrecord/ATAC/tf_0.tfrecord
Epoch2, step5, total loss:9.756
Epoch2, step10, total loss:9.745
Epoch2, step15, total loss:9.701
Epoch2, step20, total loss:9.657
./tfrecord/RNA/tf_1.tfrecord
./tfrecord/ATAC/tf_1.tfrecord
Epoch3, step5, total loss:9.242
Epoch3, step10, total loss:9.183
Epoch3, step15, total loss:9.093
./tfrecord/RNA/tf_0.tfrecord
./tfrecord/ATAC/tf_0.tfrecord
Epoch3, step5, total loss:8.775
Epoch3, step10, total loss:8.751
Epoch3, step15,

# test

In [17]:
saved_result_path = f'./result/'
if not os.path.exists(saved_result_path):
    os.makedirs(saved_result_path)
RNA_tf_path = f'./tfrecord/RNA/'
ATAC_tf_path = f'./tfrecord/ATAC/'
dict_ = {'batch_size': 128, 'epoch_pretrain': 20, 'lr': 1e-4, 'drop_rate': 0.1}
weight_path = f'./weight/pretrain/'
cell_embed_RNA_all, cell_embed_ATAC_all, RNA_id_all = attune_test(RNA_tf_path = RNA_tf_path, ATAC_tf_path = ATAC_tf_path, 
                                                                  saved_weight_path_pretrain = weight_path, n_cells_for_sample = None,
                                                                  super_parameters = dict_)
f = {'cell_embed_RNA': cell_embed_RNA_all,
     'cell_embed_ATAC': cell_embed_ATAC_all,
     'RNA_id': RNA_id_all,}
ep = dict_['epoch_pretrain']
np.savez_compressed(saved_result_path + f'./result_ep{ep}1.npz', **f)

load saved weight
./tfrecord/RNA/tf_1.tfrecord
./tfrecord/ATAC/tf_1.tfrecord
feature_len: 50000
all_samples num:128
all_samples num:256
all_samples num:384
all_samples num:512
all_samples num:640
all_samples num:768
all_samples num:896
all_samples num:1024
all_samples num:1152
all_samples num:1280
all_samples num:1408
all_samples num:1536
all_samples num:1664
all_samples num:1792
all_samples num:1920
./tfrecord/RNA/tf_0.tfrecord
./tfrecord/ATAC/tf_0.tfrecord
feature_len: 50000
all_samples num:128
all_samples num:256
all_samples num:384
all_samples num:512
all_samples num:640
all_samples num:768
all_samples num:896
all_samples num:1024
all_samples num:1152
all_samples num:1280
all_samples num:1408
all_samples num:1536
all_samples num:1664
all_samples num:1792
all_samples num:1920
all_samples num:2048
all_samples num:2176
all_samples num:2304
all_samples num:2432
all_samples num:2560
all_samples num:2688
